In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 31181115
paper_name = 'wong_arumugam_2019' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/journal.pone.0218189.s008.xlsx', sheet_name='Plan 1', skiprows=2)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4817 x 7


In [7]:
original_data['genes'] = original_data['genes'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [9]:
original_data.loc[original_data['genes']=='2001-10-01000000','genes'] = 'OCT1'

In [10]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['genes'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

             Unnamed: 0      genes         logFC    logCPM   LR  PValue  FDR  \
index_input                                                                    
4572                214  YBR160WAS  2.544544e-15  0.285609  0.0     1.0  1.0   

                  orfs  
index_input             
4572         YBR160WAS  


In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data.set_index('orfs', inplace=True)
original_data.index.name='orf'

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(4791, 6)

In [16]:
original_data.head()

,Unnamed: 0,logFC,logCPM,LR,PValue,FDR
orf,,,,,,
YAL002W,2433.0,-0.042921,4.957853,0.006289,0.936792,1.000000
YAL004W,3891.0,0.311137,8.143427,3.371866,0.066319,0.498843
YAL005C,4676.0,-0.213830,8.666428,1.157173,0.282053,0.812436
YAL007C,4085.0,-0.177573,7.945580,0.502007,0.478620,0.935963
YAL008W,557.0,-0.106706,7.923146,0.170423,0.679736,1.000000


# Prepare the final dataset

In [17]:
data = original_data[['logFC']].copy()

In [18]:
dataset_ids = [16598]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,16598
data_type,value
orf,
YAL002W,-0.042921
YAL004W,0.311137
YAL005C,-0.213830
YAL007C,-0.177573
YAL008W,-0.106706


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [23]:
data.head()

,dataset_id,16598
,data_type,value
gene_id,orf,
2,YAL002W,-0.042921
1863,YAL004W,0.311137
4,YAL005C,-0.213830
5,YAL007C,-0.177573
6,YAL008W,-0.106706


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan

In [27]:
data_all = data.join(data_norm)

In [28]:
data_all.head()

dataset_id          16598          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.042921 -0.095830
1863    YAL004W  0.311137  0.389539
4       YAL005C -0.213830 -0.330123
5       YAL007C -0.177573 -0.280420
6       YAL008W -0.106706 -0.183270

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 31181115...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.06s/it]

Updating the data_modified_on field...
